In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', None)
from utils.mlss import ivpack, aer, r_base, fixest
from utils.mlss import (
    train_lgb,
    train_random_forest,
    linear_prediction,
    quadratic_prediction,
    construct_instrument,
    cross_fit_instruments,
    split_sample_ml,
    anderson_rubin,
    feols,
    hyperparameter_search,
    linear_prediction,
    quadratic_prediction,
    quadratic_prediction_with_interaction,
    parse_ar
)

import seaborn as sns
import matplotlib.pyplot as plt
import re
import statsmodels.api as sm

C:\WBG\anaconda\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\WBG\anaconda\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\WBG\anaconda\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
Unable to determine R home: [WinError 2] The system cannot find the file specified
Unable to determine R library path: Command 

In [3]:
#df_cases = pd.read_stata("C:\\Users\\wb570559\\india_project\\monthly_mortality\\data\\tempdata_yearly_juris_pol.dta")

In [4]:
#df_cases_all = pd.DataFrame()

In [5]:
import glob
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wb570559\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wb570559\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# data prep -
# 1. yearly pollution

def cleaner(txt_file):
    
    text = txt_file
    text = re.sub(r"(&[a-zA-Z]*;)", " ", text)  # the txt files had some unwanted text like &rsquo; this line removes such text
    text = text.lower()
    
    # remove punctuation and numbers from the string
    punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~=+`|0123456789 '''  # removing punctuations except hyphens
    
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")
            
    
    text = text.replace(" st ", " ")
    text = text.replace(" nd ", " ")
    text = text.replace(" rd ", " ")
    text = text.replace(" th ", " ")
    text = text.replace("hellip", " ")
    text = text.replace("rsquo", " ")
    text = text.replace("ldquo", " ")
    text = text.replace("rdquo", " ")
    text = text.replace("ndash", " ")
    text = text.replace("--", " ")

    
    return text



def cleaner2(txt_file):
    
    text = txt_file
    text = re.sub(r"(&[a-zA-Z]*;)", " ", text)  # the txt files had some unwanted text like &rsquo; this line removes such text
    text = text.lower()
    
    # remove punctuation and numbers from the string
    punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~=+`| '''  # removing punctuations except hyphens
    
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")
            
    
    text = text.replace(" st ", " ")
    text = text.replace(" nd ", " ")
    text = text.replace(" rd ", " ")
    text = text.replace(" th ", " ")
    text = text.replace("hellip", " ")
    text = text.replace("rsquo", " ")
    text = text.replace("ldquo", " ")
    text = text.replace("rdquo", " ")
    text = text.replace("ndash", " ")
    text = text.replace("--", " ")

    
    return text

In [7]:
#1. yearly pollution

In [8]:
df_yp = pd.read_stata(r"C:\Users\wb570559\github\india_air_pollution\data\temp_data\tempdata_monthly_air_olexiy_mhml.csv")

In [9]:
df_yp.head()

,month,year,district,EnvCase,NotGreenCaseMedian,GreenCaseMedian,NotGreenCaseMean,GreenCaseMean,bc,du,oc14,oc16,oc18,pm14,pm16,pm18,so2,so4,ss,NumJudges,JudgeWoman,JudgeGrad,JudgePostGrad,GovtPetitioner,GovtRespondent,Appeal,Constitutional,lsa_vec1,lsa_vec2,lsa_vec3,lsa_vec4,lsa_vec5,lsa_vec6,lsa_vec7,lsa_vec8,lsa_vec9,lsa_vec10,lsa_vec11,lsa_vec12,lsa_vec13,lsa_vec14,lsa_vec15,lsa_vec16,lsa_vec17,lsa_vec18,lsa_vec19,lsa_vec20,lsa_vec21,lsa_vec22,lsa_vec23,lsa_vec24,lsa_vec25,Onelsa_vec1,Onelsa_vec2,Onelsa_vec3,Onelsa_vec4,Onelsa_vec5,Onelsa_vec6,Onelsa_vec7,Onelsa_vec8,Onelsa_vec9,Onelsa_vec10,Onelsa_vec11,Onelsa_vec12,Onelsa_vec13,Onelsa_vec14,Onelsa_vec15,Onelsa_vec16,Onelsa_vec17,Onelsa_vec18,Onelsa_vec19,Onelsa_vec20,Onelsa_vec21,Onelsa_vec22,Onelsa_vec23,Onelsa_vec24,Onelsa_vec25,d2v_vec1,d2v_vec2,d2v_vec3,d2v_vec4,d2v_vec5,d2v_vec6,d2v_vec7,d2v_vec8,d2v_vec9,d2v_vec10,d2v_vec11,d2v_vec12,d2v_vec13,d2v_vec14,d2v_vec15,d2v_vec16,d2v_vec17,d2v_vec18,d2v_vec19,d2v_vec20,d2v_vec21,d2v_vec22,d2v_vec23,d2v_vec24,d2v_vec25,jidentity_aadesai,jidentity_aalam,jidentity_aanand,jidentity_abmukherjee,jidentity_absrivastava,jidentity_abhan,jidentity_abhate,jidentity_acrao,jidentity_achaudhary,jidentity_achowdhrigarg,jidentity_adjain,jidentity_adsingh,jidentity_adeb,jidentity_afarooq,jidentity_ahariprasad,jidentity_akagrawal,jidentity_akbasheer,jidentity_akbasu,jidentity_akganguly,jidentity_akgoswami,jidentity_akjayasankarannambiar,jidentity_akpatnaik,jidentity_aksikri,jidentity_aksingh,jidentity_akulasekaran,jidentity_akumar,jidentity_alakshmanan,jidentity_ambasheerahamed,jidentity_amkhanwilkar,jidentity_amshaffique,jidentity_amathur,jidentity_amohunta,jidentity_amuhamedmustaque,jidentity_apdeshpande,jidentity_apsahi,jidentity_apasayat,jidentity_aqureshi,jidentity_aryousuf,jidentity_aryousufbssajwanrctriv,jidentity_aramalingam,jidentity_araman,jidentity_asanand,jidentity_asbopanna,jidentity_aschandurkar,jidentity_asgadkari,jidentity_asnaidu,jidentity_asoka,jidentity_asreddy,jidentity_asavant,jidentity_asayed,jidentity_asen,jidentity_ashah,jidentity_asikri,jidentity_asingh,jidentity_atiwari,jidentity_avramakrishnapillai,jidentity_avseshasai,jidentity_aykogje,jidentity_aakankshavyas,jidentity_abhays,jidentity_abhaysoka,jidentity_achintyamallabujorbaru,jidentity_acting,jidentity_adityakumartrivedi,jidentity_aftabalam,jidentity_ahmadi,jidentity_ahmadiam,jidentity_ahsanuddinamanullah,jidentity_ajayadeshpande,jidentity_ajaybhanot,jidentity_ajaydeshpande,jidentity_ajaykumarmittal,jidentity_ajaymohangoel,jidentity_ajayrastogi,jidentity_ajaytewari,jidentity_ajitkumarj,jidentity_ajitprakashshah,jidentity_ajitsingh,jidentity_akhileshchandra,jidentity_akilkureshi,jidentity_alexanderthomas,jidentity_alokaradhe,jidentity_alokkumarverma,jidentity_alokmathur,jidentity_aloksharma,jidentity_aloksingh,jidentity_alokverma,jidentity_altamaskabir,jidentity_amankashyap,jidentity_amitrawal,jidentity_amitavkgupta,jidentity_amitavaroy,jidentity_amolrattansingh,jidentity_amreshwarpratapsahi,jidentity_anandbyrareddy,jidentity_anandprasadsinha,jidentity_anandasen,jidentity_anantbijaysingh,jidentity_anantsdave,jidentity_anildevsingh,jidentity_anilknarendran,jidentity_anilkumarchoudhary,jidentity_anilkumarsinha,jidentity_anilkumarupadhyay,jidentity_anjanamishra,jidentity_anjanaprakash,jidentity_anjulipalo,jidentity_ankitsingla,jidentity_annapurnadixit,jidentity_anojain,jidentity_anoopchitkara,jidentity_antonydominic,jidentity_anusivaraman,jidentity_anubharawatchoudhary,jidentity_anujaprabhudessai,jidentity_anupindersinghgrewal,jidentity_apareshkumarsingh,jidentity_arramalingam,jidentity_aravindkumar,jidentity_arijitpasayat,jidentity_arunkumar,jidentity_arunmishra,jidentity_arunmonga,jidentity_arunpalli,jidentity_aruntandon,jidentity_arvindsrivastava,jidentity_ashimkumarroy,jidentity_ashokbhinchigeri,jidentity_ashokbhan,jidentity_ashokbhushan,jidentity_ashokgnijagannavar,jidentity_ashokjindal,jidentity_ashokskinagi,jidentity_ashutoshkumar,jidentity_ashutoshmohunta,ji

In [10]:
df_yp = df_yp[df_yp["year"].notnull()]

df_yp["districtXyear"] = df_yp["district"].astype(str) + "_" + df_yp["year"].astype(str) 
df_yp["year_"] = df_yp["year"].apply(lambda x : "year"+str(cleaner2(str(x))))
df_yp["month_"] = df_yp["month"].apply(lambda x : "month"+str(cleaner2(str(x))))

df_yp["district"] = df_yp["district"].apply(lambda x : str(cleaner(x)))

district_dummies = pd.get_dummies(df_yp["district"],drop_first=True)
year_dummies = pd.get_dummies(df_yp["year_"],drop_first=True)
month_dummies = pd.get_dummies(df_yp["month_"],drop_first=True)

year_cols = list(year_dummies.columns)
district_cols = list(district_dummies.columns)
month_cols = list(month_dummies.columns)

df_yp = pd.concat([df_yp,year_dummies,district_dummies,month_dummies], axis=1)



In [11]:
characteristic_cols_main = ["JudgeWoman_lag_1", "JudgePostGrad_lag_1"]

In [12]:
d2v_list = ["d2v_vec1_lag_1","d2v_vec2_lag_1","d2v_vec3_lag_1","d2v_vec4_lag_1","d2v_vec5_lag_1",
           "d2v_vec6_lag_1","d2v_vec7_lag_1","d2v_vec8_lag_1","d2v_vec9_lag_1","d2v_vec10_lag_1",
           "d2v_vec11_lag_1","d2v_vec12_lag_1","d2v_vec13_lag_1","d2v_vec14_lag_1","d2v_vec15_lag_1",
           "d2v_vec16_lag_1","d2v_vec17_lag_1","d2v_vec18_lag_1","d2v_vec19_lag_1","d2v_vec20_lag_1",
           "d2v_vec21_lag_1","d2v_vec22_lag_1","d2v_vec23_lag_1","d2v_vec24_lag_1","d2v_vec25_lag_1"]

In [13]:
lsa_list = ["lsa_vec1_lag_1","lsa_vec2_lag_1","lsa_vec3_lag_1","lsa_vec4_lag_1","lsa_vec5_lag_1",
           "lsa_vec6_lag_1","lsa_vec7_lag_1","lsa_vec8_lag_1","lsa_vec9_lag_1","lsa_vec10_lag_1",
           "lsa_vec11_lag_1","lsa_vec12_lag_1","lsa_vec13_lag_1","lsa_vec14_lag_1","lsa_vec15_lag_1",
           "lsa_vec16_lag_1","lsa_vec17_lag_1","lsa_vec18_lag_1","lsa_vec19_lag_1","lsa_vec20_lag_1",
           "lsa_vec21_lag_1","lsa_vec22_lag_1","lsa_vec23_lag_1","lsa_vec24_lag_1","lsa_vec25_lag_1"]

In [14]:
pollution_vars = [
                  "bc","du","oc14","oc16","oc18","pm14","pm16","pm18","so2","so4","ss"
                 ]

In [15]:
#vectype = ["d2v", "lsa"]
vectype = ["d2v"]

In [16]:
covs_no_dum = ["GovtRespondent_lag_1","Appeal_lag_1","Constitutional_lag_1", "CaseDummy_lag_1"] 
covs_orig = covs_no_dum+year_cols+district_cols

In [17]:
#df_yp_ji = pd.read_stata(r"C:\Users\wb570559\mlss_replication\processed_data\tempdata_yearly_juris_pol_judge_identity.dta")

In [18]:
#all_cols = list(df_yp.columns)

In [19]:
#judge_identity_vars = [ele for ele in all_cols if "jidentity" in ele]

In [20]:
#judge_identity_vars[:5]

In [21]:
#global fwl

def fit_mlss_rf(seed, covs=None):
    rng = np.random.RandomState(seed)
    sample = (rng.rand(len(df)) > 0.5).astype(int)

    state = np.random.get_state()
    np.random.seed(seed)
    data, fwl_data = split_sample_ml(
        df,
        sample,
        characteristic_cols,
        "FracGreenCases_lag_1",
        outcomevar,
        covariates=covs,
        training=train_random_forest,
        name="rf",
        tune=False,
        clf=searched_clf,
    )
    #fwl = fwl_data
    np.random.set_state(state)

    if covs is not None:
        fwl_data.index = data.index
        #print("abc")
        return analyze(fwl_data, df, sample, "rf")
    else:
        #print("pqr")
        return analyze(data, df, sample, "rf")

def analyze(data, df, sample, name):
    s0 = df.loc[sample == 0].index
    s1 = df.loc[sample == 1].index
    if "y" in data.columns:
        coef, fit = feols(f"y ~ 1 | d ~ z", data=data,)
        return {
            "coef": coef,
            "fit": fit,
            "ar1": anderson_rubin(data.loc[s0], "y ~ d", "~z",)[0],
            "ar2": anderson_rubin(data.loc[s1], "y ~ d", "~z",)[0],
        }
    else:
        coef, fit = feols(f"{outcomevar} ~ 1 | FracGreenCases_lag_1 ~ {name}_inst", data=data,)
        return {
            "coef": coef,
            "fit": fit,
            "ar1": anderson_rubin(
                data.loc[s0], f"{outcomevar} ~ FracGreenCases_lag_1", f"~{name}_inst",
            )[0],
            "ar2": anderson_rubin(
                data.loc[s1], f"{outcomevar} ~ FracGreenCases_lag_1", f"~{name}_inst",
            )[0],
            "r2": data[["FracGreenCases", f"{name}_inst"]].corr().iloc[0, 1] ** 2,
        }
  

def fit_mlss(seed, trainer, covs=None):
    rng = np.random.RandomState(seed)
    sample = (rng.rand(len(df)) > 0.5).astype(int)

    data, fwl_data = split_sample_ml(
        df,
        sample,
        characteristic_cols,
        "FracGreenCases",
        outcomevar,
        covariates=covs,
        training=trainer,
        name="lgb",
        tune=False,
    )

    if covs is not None:
        fwl_data.index = data.index
        fwl_data.to_csv("fwl.csv")
        return analyze(fwl_data, df, sample, "lgb")
    else:
        return analyze(data, df, sample, "lgb")

plot_kwargs = dict(ls="", capthick=1, capsize=4,)


def plot_result(res, y, ax=None):
    coef, wald_se = pd.DataFrame(res["coef"]).iloc[1, :2]

    numbers, shape = parse_ar(res["ar1"])
    #if shape == "interval":
    lb, ub = numbers
        
    numbers, shape = parse_ar(res["ar2"])
    #if shape == "interval":
    lb = max(lb, numbers[0])
    ub = min(ub, numbers[1])
        
    lb = max(lb, -1000)
    ub = min(ub, 1000)

    (plt if ax is None else ax).errorbar(
        y=y, x=coef, xerr=1.96 * wald_se, marker=".", **plot_kwargs, color="b"
    )
    (plt if ax is None else ax).errorbar(
        y=y + 0.1,
        x=coef,
        xerr=np.array([[coef - lb], [ub - coef]]),
        **plot_kwargs,
        color="r",
    )

    if "r2" in res:
        (plt if ax is None else ax).text(
            y=y + 0.2, x=1, s=f'{100*res["r2"]:.1f}%', fontsize="small"
        )

def fit_and_plot(seed, ax, precompute=None, **kwargs):
    if precompute is not None:
        [lgb_res, rf_res, linear_res, quad_res, quad_inter_res] = precompute
    else:
        lgb_res = fit_mlss(seed, trainer=train_lgb, **kwargs)
        rf_res = fit_mlss_rf(seed, **kwargs)
        linear_res = fit_mlss(seed, trainer=linear_prediction, **kwargs)
        quad_res = fit_mlss(seed, trainer=quadratic_prediction, **kwargs)
        quad_inter_res = fit_mlss(
            seed, trainer=quadratic_prediction_with_interaction, **kwargs
        )

    plot_result(lgb_res, 5, ax=ax)
    plot_result(rf_res, 4, ax=ax)
    plot_result(linear_res, 3, ax=ax)
    plot_result(quad_res, 2, ax=ax)
    plot_result(quad_inter_res, 1, ax=ax)

    return [lgb_res, rf_res, linear_res, quad_res, quad_inter_res]

In [22]:
faulty_ar_tsls = []
faulty_ar_tsls2 = []

for outcomevar in pollution_vars:
    #flag_ = 0
    
    suffix_ar_tsls = ""
    suffix_ar_tsls2 = ""
    for tmvect in vectype:
        #tmvect="d2v"
        
        if(tmvect=="d2v"):
            
            characteristic_cols_orig = characteristic_cols_main + d2v_list
            
        if(tmvect=="lsa"):
            
            characteristic_cols_orig = characteristic_cols_main + lsa_list
            
        df = df_yp
        
        ##########
        #shashank - 
        #y-var should be notnull while residualising
        df = df[df[outcomevar].notnull()]
        all_cols = list(df.columns)
        judge_identity_vars_all = [ele for ele in all_cols if "jidentity" in ele]
        judge_identity_vars = [ele for ele in judge_identity_vars_all if "lag" in ele]
        df_judges_temp = df[judge_identity_vars]
        df_judges_temp = df_judges_temp.fillna(0)
        df = df.fillna(df_judges_temp) # nan judge identity columns to be filled with 0
        
        faulty = []

        for ele in list(df.columns):

            desc = list(df[ele].unique())

            if len(desc) == 1:
                
                faulty.append(ele)
                
        df = df.drop(faulty, axis=1)
        df = df.reset_index(drop=True)
        meanoutcome = df[outcomevar].mean()
        stdoutcome = df[outcomevar].std()
        ##########
        
        characteristic_cols = list(pd.Series(characteristic_cols_orig)[~pd.Series(characteristic_cols_orig).isin(pd.Series(faulty))])
        covs = list(pd.Series(covs_orig)[~pd.Series(covs_orig).isin(pd.Series(faulty))])
        
        year_cols_temp = list(pd.Series(year_cols)[pd.Series(year_cols).isin(pd.Series(covs))])
        month_cols_temp = list(pd.Series(month_cols)[pd.Series(month_cols).isin(pd.Series(covs))])
        district_cols_temp = list(pd.Series(district_cols)[pd.Series(district_cols).isin(pd.Series(covs))])
        
        residualise_on = year_cols_temp+month_cols_temp+district_cols_temp
        
        # residualising - 
        
        model = sm.OLS(df[outcomevar],df[residualise_on])
        results = model.fit()
        residuals = results.resid
        
        df[outcomevar] = residuals
        
        remove_year_dist = year_cols+district_cols
        
        covs = list(pd.Series(covs)[~pd.Series(covs).isin(pd.Series(remove_year_dist))])
        
        #df2 = df_yp_ji
        
        df2 = df
        
        ##########
        #shashank - 
        #y-var should be notnull while residualising
        df2 = df2[df2[outcomevar].notnull()]
        df2 = df2.reset_index(drop=True)
        #meanoutcome = df2[outcomevar].mean()
        #stdoutcome = df2[outcomevar].std()
        all_cols = list(df2.columns)
        judge_identity_vars = [ele for ele in all_cols if "jidentity" in ele]
        characteristic_cols2 = judge_identity_vars
        ##########
        
        for c in characteristic_cols:
            df[c + "2"] = df[c] ** 2
        quadratics = [c + "2" for c in characteristic_cols]
        
        for c in characteristic_cols2:
            df2[c + "2"] = df2[c] ** 2
        quadratics2 = [c + "2" for c in characteristic_cols2]
        
        
        tsls_simple = feols(
            f"{outcomevar} ~ 1 | FracGreenCases ~ {'+'.join(characteristic_cols)}", data=df,
        )[0]
        
        tsls_simple2 = feols(
            f"{outcomevar} ~ 1 | FracGreenCases ~ {'+'.join(characteristic_cols2)}", data=df2,
        )[0]
        
        ar_tsls_simple = anderson_rubin(
            df, f"{outcomevar} ~ FracGreenCases", f"~{'+'.join(characteristic_cols)}",
        )[0]
        
        ar_tsls_simple2 = anderson_rubin(
            df2, f"{outcomevar} ~ FracGreenCases", f"~{'+'.join(characteristic_cols2)}",
        )[0]

        tsls_simple_quad = feols(
            f"{outcomevar} ~ 1 | FracGreenCases ~ {'+'.join(characteristic_cols + quadratics)}",
            data=df,
        )[0]
        
        ar_tsls_simple_quad = anderson_rubin(
            df,
            f"{outcomevar} ~ FracGreenCases",
            f"~{'+'.join(characteristic_cols + quadratics)}",
        )[0]
        

        
        
        ##
        print(feols(
            f"{outcomevar} ~ 1 | FracGreenCases ~ {'+'.join(characteristic_cols + quadratics)}",
            data=df,
        )[1])

        tsls = feols(
            f"{outcomevar} ~ {'+'.join(covs)} | FracGreenCases ~ {'+'.join(characteristic_cols)}",
            data=df,
        )[0]
        
        tsls2 = feols(
            f"{outcomevar} ~ {'+'.join(covs)} | FracGreenCases ~ {'+'.join(characteristic_cols2)}",
            data=df2,
        )[0]
        


        #print(tsls.T.iloc[:2, 1])

        ar_tsls = anderson_rubin(
            df,
            f"{outcomevar} ~ FracGreenCases + {'+'.join(covs)}",
            f"~{'+'.join(characteristic_cols)} + {'+'.join(covs)}",
        )[0]
        
        ar_tsls2 = anderson_rubin(
            df2,
            f"{outcomevar} ~ FracGreenCases + {'+'.join(covs)}",
            f"~{'+'.join(characteristic_cols2)} + {'+'.join(covs)}",
        )[0]
        
        
        tsls_quad = feols(
            f"{outcomevar} ~ {'+'.join(covs)} | FracGreenCases ~ {'+'.join(characteristic_cols + quadratics)}",
            data=df,
        )[0]
        
        print(
            feols(
                f"{outcomevar} ~ {'+'.join(covs)} | FracGreenCases ~ {'+'.join(characteristic_cols + quadratics)}",
                data=df,
            )[1]
        )
         
            
        ar_quad = anderson_rubin(
            df,
            f"{outcomevar} ~ FracGreenCases + {'+'.join(covs)}",
            f"~{'+'.join(characteristic_cols + quadratics)} + {'+'.join(covs)}",
        )[0]
        
        
        #df2 = df[df["{outcomevar}"].notnull()]
        df = df.reset_index(drop = True)
        import numpy as np
        searched_clf, best_params = hyperparameter_search(
            df.sample(random_state=np.random.RandomState(123), frac=0.5),
            characteristic_cols,
            "FracGreenCases",
        )

        
        s1 = fit_and_plot(123, ax=None, covs=covs)
        s2 = fit_and_plot(321, ax=None, covs=covs)
        s3 = fit_and_plot(42, ax=None, covs=covs)
        
        #if (ar_tsls != "Empty Set" and ar_tsls2 != "Empty Set"):


        # plot 2 - 

        f, axs = plt.subplots(ncols=3, figsize=(10, 4), sharey=True, sharex=True)

        fit_and_plot(123, ax=axs[0], precompute=s1, covs=covs)
        fit_and_plot(321, ax=axs[1], precompute=s2, covs=covs)
        fit_and_plot(42, ax=axs[2], precompute=s3, covs=covs)

        est, wald = pd.DataFrame(tsls).iloc[1, :2]
        if (ar_tsls != "Empty Set"):
            lb, ub = parse_ar(ar_tsls)[0]
            lb = max(lb, -1000)
            ub = min(ub, 1000)
        else:
            print("#########----------ar_tsls empty!--------###########")
            #flag_ = 1
            faulty_ar_tsls.append(outcomevar)
            lb = 1000
            ub = 1000
            suffix_ar_tsls = "_ar_tsls_empty_"

        axs[0].errorbar(
            y=7, x=est, xerr=1.96 * wald, marker=".", **plot_kwargs, color="b", label="Wald"
        )
        axs[0].errorbar(
            y=7 + 0.1,
            x=est,
            xerr=np.array([[est - lb], [ub - est]]),
            **plot_kwargs,
            color="r",
            label="Anderson-Rubin",
        )

        est, wald = pd.DataFrame(tsls_quad).iloc[1, :2]
        axs[0].errorbar(y=6, x=est, xerr=1.96 * wald, marker=".", **plot_kwargs, color="b")

        try:
            lb, ub = parse_ar(ar_quad)[0]
            lb = max(lb, -1000)
            ub = min(ub, 1000)
            axs[0].errorbar(
                y=6 + 0.1,
                x=est,
                xerr=np.array([[est - lb], [ub - est]]),
                **plot_kwargs,
                color="r",
            )
        except:
            print("passing----------------------------")
            pass

##### Judge Identity - 

        est, wald = pd.DataFrame(tsls2).iloc[1, :2]
        if (ar_tsls2 != "Empty Set"): 
            lb, ub = parse_ar(ar_tsls2)[0]
            lb = max(lb, -1000)
            ub = min(ub, 1000)
        else:
            print("#########----------ar_tsls2 empty!--------###########")
            #flag_ = 1
            faulty_ar_tsls2.append(outcomevar)
            lb = 1000
            ub = 1000
            suffix_ar_tsls2 = "_ar_tsls2_empty_"

        axs[0].errorbar(
            y=8, x=est, xerr=1.96 * wald, marker=".", **plot_kwargs, color="g", label="Wald (Judge Identity)"
        )
        axs[0].errorbar(
            y=8 + 0.1,
            x=est,
            xerr=np.array([[est - lb], [ub - est]]),
            **plot_kwargs,
            color="y",
            label="Anderson-Rubin (Judge Identity)",
        )

#####

        plt.yticks(
            ticks=list(range(1, 9)),
            labels=[
                "Saturated Quadratic",
                "Quadratic",
                "Linear",
                "Random Forest",
                "LightGBM",
                "TSLS (Quadratic)",
                "TSLS",
                "TSLS (Judge Identity)"
            ],
        )

        for i in [0, 1, 2]:
            axs[i].axvline(0, color="k", ls="--")

        axs[1].set_xlabel(f"Estimated {outcomevar} (Monthly) (with covariates) ({tmvect})")

        axs[0].legend(loc=(3.7, 0), frameon=False)
        plt.xlim((-meanoutcome-stdoutcome), (meanoutcome+stdoutcome))
        fig_name = str(outcomevar) + "_" + str(tmvect) + suffix_ar_tsls + suffix_ar_tsls2 + "_with_cov_monthly_lagby_1.pdf"
        

        savepath = os.path.join(r"C:\Users\wb570559\github\india_air_pollution\mhml\results", fig_name)
        plt.savefig(savepath, bbox_inches="tight")

KeyError: 'JudgeWoman_lag_1'

In [ ]:
ar_tsls2


In [ ]:
import PyPDF2
from PyPDF2 import PdfFileMerger, PdfFileReader
import glob
import os

In [ ]:
paths = glob.glob(r"C:\Users\wb570559\mlss_replication\air_pollution\results\*.pdf")

In [ ]:
paths.sort()

In [ ]:
#paths

In [ ]:
"""# Call the PdfFileMerger
mergedObject = PdfFileMerger()
 
# I had 116 files in the folder that had to be merged into a single document
# Loop through all of them and append their pages
for fileName in paths:
    mergedObject.append(PdfFileReader(fileName, 'rb'))
 
 # Write all the files into a file which is named as shown below
mergedObject.write("mergedfilesoutput_final_pollution.pdf")"""

In [ ]:
#!pip install --user pandas --upgrade

In [ ]:
df2.head()